In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
%load_ext autoreload
%autoreload 2

/home/ffisin/PycharmProjects/dialogueBotTF/venv/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
from dialogue_bot.model import Model
config = {
    'vocab_path': '../vocabs/vocab.txt',
    'context_size': 2,
    'embedding_params': {
        'vocab_size': 87,
        'embedding_size': 50,
    },
    'encoder_params': {
        'hidden_size': 100,
        'context_size': 2
    },
    'decoder_params': {
        'embedding_size': 64,
        'vocab_size': 87,
        'hidden_size': 77,
        'output_vocab_size': 87,
    }
}

from dialogue_bot.dataset.dataset import get_dataset
dataset = get_dataset(
    data_path='TlkPersonaChatRus/dialogues.tsv',
    context_window_size=2, 
    batch_size=30,
)

In [10]:
def calculate_normalization_loss(
        normalization_logits: tf.Tensor,
        expected_normalization_labels: tf.Tensor,
):
    non_pad_mask = tf.not_equal(
        x=expected_normalization_labels,
        y=0,
    )
    weights = tf.cast(non_pad_mask, tf.float32)
    return tfa.seq2seq.sequence_loss(
        logits=normalization_logits,
        targets=expected_normalization_labels,
        weights=weights
    )

In [14]:
def train(model, optimizer, clip_norm=2):
    for i, batch in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits, expected_output = model.call(batch, mode='train')
            loss = calculate_normalization_loss(logits.rnn_output, expected_output)
        gradients = tape.gradient(
            target=loss,
            sources=model.trainable_variables
        )
        gradients_norm = tf.linalg.global_norm(gradients)
        clipped_gradients = [(tf.clip_by_norm(grad, clip_norm))
                             for grad in gradients]
        clipped_gradients_norm = tf.linalg.global_norm(clipped_gradients)
        optimizer.apply_gradients(zip(clipped_gradients,
                                      model.trainable_variables))
        if i % 5 == 0:
            print(loss)

In [15]:
model = Model(params=config)
optimizer = tf.keras.optimizers.Adam()

In [16]:
train(model, optimizer)

tf.Tensor(4.4664006, shape=(), dtype=float32)
tf.Tensor(4.4337616, shape=(), dtype=float32)
tf.Tensor(4.3786836, shape=(), dtype=float32)
tf.Tensor(4.2333717, shape=(), dtype=float32)


KeyboardInterrupt: 